# Overview of Changes

This pipeline will do away with binning based on magnitude. Besides that, the survey property portion of the correction will be identical. Relative probabilities will also be calculated based on object properties. From there, the total conditional probability will just be the product of these two conditional probabilities that depend on fewer inputs (the function will act like a tensor product from 561). Object properties that get used need to be available both for Balrog objects and general gold catalog objects. These properties will then go through PCA just like the survey properties. This puts assumptions in place that other properties don't play a role. RA and DEC will not be used as these are not really properties of the object itself and could easily have overlap with survey properties. The object properties that will be used are (note that more properties will be read in such as class, RA, DEC, flags, ID numbers, etc., but these will not really be inputs to the probability function):

meas_cm_mag/SOF_CM_MAG_band, meas_psf_mag/SOF_PSF_MAG_band, meas_cm_T/SOF_CM_T, corresponding error term for size, and more to come if necessary.

Right now, the only correction that will be performed is a galaxy contamination correction as the detection rate correction was already working well. For this correction, only two probability functions need to be calculated: $P\left(O_S|T_S\right)$, and $P\left(O_G|T_G\right)$. Once these are calculated in terms of survey conditions and object properties, we will have the following multiplicative correction for observed stars:

$$\frac{P\left(O_G|T_G\right)}{P\left(O_S|T_S\right) + P\left(O_G|T_G\right) - 1}$$

The following multiplicative correction will be applied to observed galaxies:

$$\frac{P\left(O_G|T_G\right) - 1}{P\left(O_S|T_S\right) + P\left(O_G|T_G\right) - 1}$$

The key piece to this will be seeing if making these probability functions dependent on object properties eliminates the issue of getting negative counts for stars in certain regions of the sky.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import fitsio
import astropy.io.fits as fits
from astropy.table import Table
import healpy as hp
from GetObjectProperties import *
from TrainAndExtend import *
import Config

In [2]:
minGR = -0.3 # Minimum G-R color
maxGR = 1 # Maximum G-R color
res = 512 # Healpixel resolution
perCovered = 0.6 # Percent of healpixel that must have valid survey properties at the 4096 scale
sigma = 0.5 # Sigma used for gaussian weighting
perVar = 0.98 # Percent of the variance to be captured
perMap = 0.625 # Percent of the PC maps to use, adjust this later
numBins = 100 # Number of points in interpolation

In [3]:
conditions = Config.conditions
oldGalaDir = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/GalaxyContamination/'

In [4]:
deepFiles = ['/hdfs/bechtol/balrog/y3_deep_fields/y3_deep_fields_catalog/deepfields_000001.fits', 
             '/hdfs/bechtol/balrog/y3_deep_fields/y3_deep_fields_catalog/deepfields_000002.fits', 
             '/hdfs/bechtol/balrog/y3_deep_fields/y3_deep_fields_catalog/deepfields_000003.fits',
             '/hdfs/bechtol/balrog/y3_deep_fields/y3_deep_fields_catalog/deepfields_000004.fits',
             '/hdfs/bechtol/balrog/y3_deep_fields/y3_deep_fields_catalog/deepfields_000005.fits']

In [5]:
newGalaDir = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/ObjectPropertyGalaxyContamination/'

In [6]:
balrFile = '/hdfs/bechtol/balrog/y3/balrog_detection_catalog_sof_y3-merged_v1.2.fits'
measBalrFile = '/hdfs/bechtol/balrog/y3/balrog_matched_catalog_sof_y3-merged_v1.2.fits'
galaFile = newGalaDir + 'Blue_Galaxies.fits'

In [7]:
balrStarFile = '/afs/hep.wisc.edu/bechtol-group/MegansThings/balrog_detection_catalog_sof_run2_stars_v1.4_avg_added_match_flags.fits'
measBalrStarFile = '/afs/hep.wisc.edu/bechtol-group/MegansThings/balrog_matched_catalog_sof_run2_stars_v1.4.fits'
starFile = newGalaDir + 'Blue_Stars.fits'

### Location of Valid Pixels and Survey Conditions

This was calculated in a different piece of software, namely in the GalaxyContamination folder.

In [8]:
origCondFiles = Config.files[:-1]
stelFile = Config.files[-1]
pixFile = oldGalaDir + 'Valid_'+str(res)+'_Pixels.fits'
condFiles = []
for cond in conditions:
    condFiles.append(oldGalaDir + str(res) + '_' + cond + '.fits')
condFiles = np.array(condFiles)

In [9]:
# getGalaxies(minGR, maxGR, deepFiles, measBalrFile, balrFile, galaFile)

In [10]:
# getStars(minGR, maxGR, measBalrStarFile, balrStarFile, starFile)

## Two Potential Methods

There are multiple potential methods that could be used for a correction that takes survey and object properties into mind. One of the two methods is mentioned at the beginning, where the two corrections would be assumed to be independent, so they're found independently and applied one after the other. Another method would be to simply treat the object properties in the same vein as survey properties and figure out one single correction that relies on all of these. Both of these algorithmic choices have pros and cons.

The initial, separate corrections method that was described has the definite pro of being significantly faster to apply to the full sky once training and extending the survey property corrections to the full map is complete. This is because the survey property correction actually can be extended to the full map since the correction does not depend on object properties. This is not the case for the one overarching correction, in which the correction is not separated into survey and object pieces, so no extension can be made. This means that when applying the correction, it must all be done at the time of the correction itself, by taking the object and survey properties from each star and galaxy, performing the same PCA procedure, and applying the trained correction. This is much more time consuming than just doing this for the far fewer object properties that are being used.

The other main difference is in the robustness of the method. In the overarching correction, correlations between object properties and survey properties are not an issue as PCA is applied to orthogonalize all of these properties. However, in the case of separate corrections, a correlation could lead to an overcorrection since the same correction would effectively be applied twice.

Algorithmically, it seems to me that the one overarching correction would be easier to apply, but I'm not really sure about this, and either way it would not be too terribly complicated.

## Cropping Values

Values were cropped, so there are no magnitudes of 37.5. For stars, size was cropped at 2 and size error was cropped at 1. For galaxies, size was cropped at 90 and size error was cropped at 20. These were done to keep over 99% from each crop.

In [11]:
starTrainFile = newGalaDir + 'Star_Train.fits'
galaTrainFile = newGalaDir + 'Gala_Train.fits'

## Train Stars

In [12]:
# singleCorrectionTrain(starFile, condFiles, pixFile, starTrainFile, sigma, perMap, perVar, numBins, res, True)

## Train Galaxies

In [13]:
# singleCorrectionTrain(galaFile, condFiles, pixFile, galaTrainFile, sigma, perMap, perVar, numBins, res, False)

In [14]:
galas = fitsio.read(galaFile)

In [15]:
len(galas)

4381987

In [16]:
stars = fitsio.read(starFile)

In [17]:
len(stars)

541275

## Get the Real Stars and Galaxies

In [18]:
import healsparse as hsp
import skyproj
from os import listdir

In [19]:
dirStars = '/hdfs/bechtol/balrog/y3/y3a2_gold_v2p2_skim/healpixel2/'
files = listdir(dirStars)
croppedStarsDirectory = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/ObjectPropertyGalaxyContamination/GoldData/Stars/'
croppedGalasDirectory = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/ObjectPropertyGalaxyContamination/GoldData/Galaxies/'
equalSizeFileDir = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/ObjectPropertyGalaxyContamination/GoldData/EqualSize/'
usedCols = ['RA', 'DEC', 'FLAGS_FOREGROUND', 'FLAGS_BADREGIONS', 'FLAGS_FOOTPRINT', 'EXTENDED_CLASS_SOF', 'SOF_PSF_MAG_G', 'SOF_PSF_MAG_R', 'SOF_PSF_MAG_I', 'SOF_PSF_MAG_Z', 'SOF_CM_MAG_G', 'SOF_CM_MAG_R', 'SOF_CM_MAG_I', 'SOF_CM_MAG_Z', 'SOF_CM_T', 'SOF_CM_T_ERR']
validPix = fitsio.read(pixFile)['PIXEL']
pixCheck = np.full(12*(res**2), False, dtype = bool)
pixCheck[validPix] = True

In [20]:
# galaCount = 0
# starCount = 0

# for i in range(len(files)):
    
#     obsData = fitsio.read(dirStars + files[i], columns = usedCols)
    
#     RA = obsData[usedCols[0]]
#     DEC = obsData[usedCols[1]]
    
#     PIX = hp.ang2pix(res, RA, DEC, lonlat = True, nest = True)
    
#     FOREGROUND = obsData[usedCols[2]]
#     BADREGIONS = obsData[usedCols[3]]
#     FOOTPRINT = obsData[usedCols[4]]
    
#     CLASS = obsData[usedCols[5]]
    
#     GMAG_PSF = obsData[usedCols[6]]
#     RMAG_PSF = obsData[usedCols[7]]
#     IMAG_PSF = obsData[usedCols[8]]
#     ZMAG_PSF = obsData[usedCols[9]]
#     GMAG_CM = obsData[usedCols[10]]
#     RMAG_CM = obsData[usedCols[11]]
#     IMAG_CM = obsData[usedCols[12]]
#     ZMAG_CM = obsData[usedCols[13]]
#     SIZE = obsData[usedCols[14]]
#     SIZE_ERR = obsData[usedCols[15]]

#     magCut = 37
    
#     starCut = np.where((CLASS <= 1) &
#                        (CLASS >= 0) &
#                        (FOREGROUND == 0) & 
#                        (BADREGIONS < 2) & 
#                        (FOOTPRINT == 1) &
#                        (pixCheck[PIX]) &
#                        (GMAG_PSF < magCut) & 
#                        (RMAG_PSF < magCut) &
#                        (IMAG_PSF < magCut) &
#                        (ZMAG_PSF < magCut) &
#                        (GMAG_CM < magCut) &
#                        (RMAG_CM < magCut) &
#                        (IMAG_CM < magCut) &
#                        (ZMAG_CM < magCut) &
#                        (GMAG_PSF - RMAG_PSF >= minGR) &
#                        (GMAG_PSF - RMAG_PSF <= maxGR))[0]
    
#     galaCut = np.where((CLASS <= 3) &
#                        (CLASS >= 2) &
#                        (FOREGROUND == 0) & 
#                        (BADREGIONS < 2) & 
#                        (FOOTPRINT == 1) &
#                        (pixCheck[PIX]) &
#                        (GMAG_PSF < magCut) & 
#                        (RMAG_PSF < magCut) &
#                        (IMAG_PSF < magCut) &
#                        (ZMAG_PSF < magCut) &
#                        (GMAG_CM < magCut) &
#                        (RMAG_CM < magCut) &
#                        (IMAG_CM < magCut) &
#                        (ZMAG_CM < magCut) &
#                        (GMAG_CM - RMAG_CM >= minGR) &
#                        (GMAG_CM - RMAG_CM <= maxGR))[0]

#     starCount += len(starCut)
#     galaCount += len(galaCut)
    
#     star_table =  Table()
#     star_table['PIX'] = PIX[starCut]
#     star_table['GMAG_PSF'] = GMAG_PSF[starCut]
#     star_table['RMAG_PSF'] = RMAG_PSF[starCut]
#     star_table['IMAG_PSF'] = IMAG_PSF[starCut]
#     star_table['ZMAG_PSF'] = ZMAG_PSF[starCut]
#     star_table['GMAG_CM'] = GMAG_CM[starCut]
#     star_table['RMAG_CM'] = RMAG_CM[starCut]
#     star_table['IMAG_CM'] = IMAG_CM[starCut]
#     star_table['ZMAG_CM'] = ZMAG_CM[starCut]
#     star_table['SIZE'] = SIZE[starCut]
#     star_table['SIZE_ERR'] = SIZE_ERR[starCut]
#     writeToStarFile = croppedStarsDirectory + ("0" * (4 - len(str(i)))) + str(i) + '.fits'
#     star_table.write(writeToStarFile, overwrite = True)
    
#     gala_table =  Table()
#     gala_table['PIX'] = PIX[galaCut]
#     gala_table['GMAG_PSF'] = GMAG_PSF[galaCut]
#     gala_table['RMAG_PSF'] = RMAG_PSF[galaCut]
#     gala_table['IMAG_PSF'] = IMAG_PSF[galaCut]
#     gala_table['ZMAG_PSF'] = ZMAG_PSF[galaCut]
#     gala_table['GMAG_CM'] = GMAG_CM[galaCut]
#     gala_table['RMAG_CM'] = RMAG_CM[galaCut]
#     gala_table['IMAG_CM'] = IMAG_CM[galaCut]
#     gala_table['ZMAG_CM'] = ZMAG_CM[galaCut]
#     gala_table['SIZE'] = SIZE[galaCut]
#     gala_table['SIZE_ERR'] = SIZE_ERR[galaCut]
#     writeToGalaFile = croppedGalasDirectory + ("0" * (4 - len(str(i)))) + str(i) + '.fits'
#     gala_table.write(writeToGalaFile, overwrite = True)

In [21]:
totStar = 20297262
totGala = 188368270
totProc = 5000

starFileCount = np.round(totStar / (totGala + totStar) * totProc).astype(int)
galaFileCount = np.round(totGala / (totGala + totStar) * totProc).astype(int)

starPerFile = np.ceil(totStar / starFileCount).astype(int)
galaPerFile = np.ceil(totGala / galaFileCount).astype(int)

In [22]:
testArray = np.arange(47)
take = 5
while len(testArray) > take:
    small = testArray[0:5]
    testArray = np.delete(testArray, np.arange(5))
    print(small)
excess = testArray
excess = np.append(excess, np.array([47, 48, 49]))
print(excess)
excess = np.delete(excess, np.arange(5))
print(len(excess))

[0 1 2 3 4]
[5 6 7 8 9]
[10 11 12 13 14]
[15 16 17 18 19]
[20 21 22 23 24]
[25 26 27 28 29]
[30 31 32 33 34]
[35 36 37 38 39]
[40 41 42 43 44]
[45 46 47 48 49]
0


In [23]:
# starIndex = 0
# galaIndex = 0

# starPIXExtra = np.array([])
# starGMAG_PSFExtra = np.array([])
# starRMAG_PSFExtra = np.array([])
# starIMAG_PSFExtra = np.array([])
# starZMAG_PSFExtra = np.array([])
# starGMAG_CMExtra = np.array([])
# starRMAG_CMExtra = np.array([])
# starIMAG_CMExtra = np.array([])
# starZMAG_CMExtra = np.array([])
# starSIZEExtra = np.array([])
# starSIZE_ERRExtra = np.array([])

# galaPIXExtra = np.array([])
# galaGMAG_PSFExtra = np.array([])
# galaRMAG_PSFExtra = np.array([])
# galaIMAG_PSFExtra = np.array([])
# galaZMAG_PSFExtra = np.array([])
# galaGMAG_CMExtra = np.array([])
# galaRMAG_CMExtra = np.array([])
# galaIMAG_CMExtra = np.array([])
# galaZMAG_CMExtra = np.array([])
# galaSIZEExtra = np.array([])
# galaSIZE_ERRExtra = np.array([])

# for i in range(len(files)):
    
#     obsData = fitsio.read(dirStars + files[i], columns = usedCols)
    
#     RA = obsData[usedCols[0]]
#     DEC = obsData[usedCols[1]]
    
#     PIX = hp.ang2pix(res, RA, DEC, lonlat = True, nest = True)
    
#     FOREGROUND = obsData[usedCols[2]]
#     BADREGIONS = obsData[usedCols[3]]
#     FOOTPRINT = obsData[usedCols[4]]
    
#     CLASS = obsData[usedCols[5]]
    
#     GMAG_PSF = obsData[usedCols[6]]
#     RMAG_PSF = obsData[usedCols[7]]
#     IMAG_PSF = obsData[usedCols[8]]
#     ZMAG_PSF = obsData[usedCols[9]]
#     GMAG_CM = obsData[usedCols[10]]
#     RMAG_CM = obsData[usedCols[11]]
#     IMAG_CM = obsData[usedCols[12]]
#     ZMAG_CM = obsData[usedCols[13]]
#     SIZE = obsData[usedCols[14]]
#     SIZE_ERR = obsData[usedCols[15]]

#     magCut = 37
    
#     starCut = np.where((CLASS <= 1) &
#                        (CLASS >= 0) &
#                        (FOREGROUND == 0) & 
#                        (BADREGIONS < 2) & 
#                        (FOOTPRINT == 1) &
#                        (pixCheck[PIX]) &
#                        (GMAG_PSF < magCut) & 
#                        (RMAG_PSF < magCut) &
#                        (IMAG_PSF < magCut) &
#                        (ZMAG_PSF < magCut) &
#                        (GMAG_CM < magCut) &
#                        (RMAG_CM < magCut) &
#                        (IMAG_CM < magCut) &
#                        (ZMAG_CM < magCut) &
#                        (GMAG_PSF - RMAG_PSF >= minGR) &
#                        (GMAG_PSF - RMAG_PSF <= maxGR))[0]
    
#     galaCut = np.where((CLASS <= 3) &
#                        (CLASS >= 2) &
#                        (FOREGROUND == 0) & 
#                        (BADREGIONS < 2) & 
#                        (FOOTPRINT == 1) &
#                        (pixCheck[PIX]) &
#                        (GMAG_PSF < magCut) & 
#                        (RMAG_PSF < magCut) &
#                        (IMAG_PSF < magCut) &
#                        (ZMAG_PSF < magCut) &
#                        (GMAG_CM < magCut) &
#                        (RMAG_CM < magCut) &
#                        (IMAG_CM < magCut) &
#                        (ZMAG_CM < magCut) &
#                        (GMAG_CM - RMAG_CM >= minGR) &
#                        (GMAG_CM - RMAG_CM <= maxGR))[0]
    
#     starPIX = PIX[starCut]
#     starGMAG_PSF = GMAG_PSF[starCut]
#     starRMAG_PSF = RMAG_PSF[starCut]
#     starIMAG_PSF = IMAG_PSF[starCut]
#     starZMAG_PSF = ZMAG_PSF[starCut]
#     starGMAG_CM = GMAG_CM[starCut]
#     starRMAG_CM = RMAG_CM[starCut]
#     starIMAG_CM = IMAG_CM[starCut]
#     starZMAG_CM = ZMAG_CM[starCut]
#     starSIZE = SIZE[starCut]
#     starSIZE_ERR = SIZE_ERR[starCut]
    
#     galaPIX = PIX[galaCut]
#     galaGMAG_PSF = GMAG_PSF[galaCut]
#     galaRMAG_PSF = RMAG_PSF[galaCut]
#     galaIMAG_PSF = IMAG_PSF[galaCut]
#     galaZMAG_PSF = ZMAG_PSF[galaCut]
#     galaGMAG_CM = GMAG_CM[galaCut]
#     galaRMAG_CM = RMAG_CM[galaCut]
#     galaIMAG_CM = IMAG_CM[galaCut]
#     galaZMAG_CM = ZMAG_CM[galaCut]
#     galaSIZE = SIZE[galaCut]
#     galaSIZE_ERR = SIZE_ERR[galaCut]
    
#     if (len(starPIXExtra) > 0) & (len(starPIXExtra) < starPerFile):
#         if len(starPIXExtra) + len(starPIX) >= starPerFile:
#             starsToAdd = starPerFile - len(starPIXExtra)
#             starPIXExtra = np.append(starPIXExtra, starPIX[0:starsToAdd])
#             starGMAG_PSFExtra = np.append(starGMAG_PSFExtra, starGMAG_PSF[0:starsToAdd])
#             starRMAG_PSFExtra = np.append(starRMAG_PSFExtra, starRMAG_PSF[0:starsToAdd])
#             starIMAG_PSFExtra = np.append(starIMAG_PSFExtra, starIMAG_PSF[0:starsToAdd])
#             starZMAG_PSFExtra = np.append(starZMAG_PSFExtra, starZMAG_PSF[0:starsToAdd])
#             starGMAG_CMExtra = np.append(starGMAG_CMExtra, starGMAG_CM[0:starsToAdd])
#             starRMAG_CMExtra = np.append(starRMAG_CMExtra, starRMAG_CM[0:starsToAdd])
#             starIMAG_CMExtra = np.append(starIMAG_CMExtra, starIMAG_CM[0:starsToAdd])
#             starZMAG_CMExtra = np.append(starZMAG_CMExtra, starZMAG_CM[0:starsToAdd])
#             starSIZEExtra = np.append(starSIZEExtra, starSIZE[0:starsToAdd])
#             starSIZE_ERRExtra = np.append(starSIZE_ERRExtra, starSIZE_ERR[0:starsToAdd])

#             star_table =  Table()
#             star_table['PIX'] = starPIXExtra
#             star_table['GMAG_PSF'] = starGMAG_PSFExtra
#             star_table['RMAG_PSF'] = starRMAG_PSFExtra
#             star_table['IMAG_PSF'] = starIMAG_PSFExtra
#             star_table['ZMAG_PSF'] = starZMAG_PSFExtra
#             star_table['GMAG_CM'] = starGMAG_CMExtra
#             star_table['RMAG_CM'] = starRMAG_CMExtra
#             star_table['IMAG_CM'] = starIMAG_CMExtra
#             star_table['ZMAG_CM'] = starZMAG_CMExtra
#             star_table['SIZE'] = starSIZEExtra
#             star_table['SIZE_ERR'] = starSIZE_ERRExtra
#             writeToStarFile = equalSizeFileDir + "Stars" + ("0" * (4 - len(str(starIndex)))) + str(starIndex) + '.fits'
#             star_table.write(writeToStarFile, overwrite = True)
#             starIndex += 1

#             starPIX = np.delete(starPIX, np.arange(starsToAdd))
#             starGMAG_PSF = np.delete(starGMAG_PSF, np.arange(starsToAdd))
#             starRMAG_PSF = np.delete(starRMAG_PSF, np.arange(starsToAdd))
#             starIMAG_PSF = np.delete(starIMAG_PSF, np.arange(starsToAdd))
#             starZMAG_PSF = np.delete(starZMAG_PSF, np.arange(starsToAdd))
#             starGMAG_CM = np.delete(starGMAG_CM, np.arange(starsToAdd))
#             starRMAG_CM = np.delete(starRMAG_CM, np.arange(starsToAdd))
#             starIMAG_CM = np.delete(starIMAG_CM, np.arange(starsToAdd))
#             starZMAG_CM = np.delete(starZMAG_CM, np.arange(starsToAdd))
#             starSIZE = np.delete(starSIZE, np.arange(starsToAdd))
#             starSIZE_ERR = np.delete(starSIZE_ERR, np.arange(starsToAdd))
#         else:
#             starsToAdd = len(starPIX)
#             starPIXExtra = np.append(starPIXExtra, starPIX[0:starsToAdd])
#             starGMAG_PSFExtra = np.append(starGMAG_PSFExtra, starGMAG_PSF[0:starsToAdd])
#             starRMAG_PSFExtra = np.append(starRMAG_PSFExtra, starRMAG_PSF[0:starsToAdd])
#             starIMAG_PSFExtra = np.append(starIMAG_PSFExtra, starIMAG_PSF[0:starsToAdd])
#             starZMAG_PSFExtra = np.append(starZMAG_PSFExtra, starZMAG_PSF[0:starsToAdd])
#             starGMAG_CMExtra = np.append(starGMAG_CMExtra, starGMAG_CM[0:starsToAdd])
#             starRMAG_CMExtra = np.append(starRMAG_CMExtra, starRMAG_CM[0:starsToAdd])
#             starIMAG_CMExtra = np.append(starIMAG_CMExtra, starIMAG_CM[0:starsToAdd])
#             starZMAG_CMExtra = np.append(starZMAG_CMExtra, starZMAG_CM[0:starsToAdd])
#             starSIZEExtra = np.append(starSIZEExtra, starSIZE[0:starsToAdd])
#             starSIZE_ERRExtra = np.append(starSIZE_ERRExtra, starSIZE_ERR[0:starsToAdd])

#             starPIX = np.delete(starPIX, np.arange(starsToAdd))
#             starGMAG_PSF = np.delete(starGMAG_PSF, np.arange(starsToAdd))
#             starRMAG_PSF = np.delete(starRMAG_PSF, np.arange(starsToAdd))
#             starIMAG_PSF = np.delete(starIMAG_PSF, np.arange(starsToAdd))
#             starZMAG_PSF = np.delete(starZMAG_PSF, np.arange(starsToAdd))
#             starGMAG_CM = np.delete(starGMAG_CM, np.arange(starsToAdd))
#             starRMAG_CM = np.delete(starRMAG_CM, np.arange(starsToAdd))
#             starIMAG_CM = np.delete(starIMAG_CM, np.arange(starsToAdd))
#             starZMAG_CM = np.delete(starZMAG_CM, np.arange(starsToAdd))
#             starSIZE = np.delete(starSIZE, np.arange(starsToAdd))
#             starSIZE_ERR = np.delete(starSIZE_ERR, np.arange(starsToAdd))

            
#     if (len(galaPIXExtra) > 0) & (len(galaPIXExtra) < galaPerFile):
#         if len(galaPIXExtra) + len(galaPIX) >= galaPerFile:
#             galasToAdd = galaPerFile - len(galaPIXExtra)
#             galaPIXExtra = np.append(galaPIXExtra, galaPIX[0:galasToAdd])
#             galaGMAG_PSFExtra = np.append(galaGMAG_PSFExtra, galaGMAG_PSF[0:galasToAdd])
#             galaRMAG_PSFExtra = np.append(galaRMAG_PSFExtra, galaRMAG_PSF[0:galasToAdd])
#             galaIMAG_PSFExtra = np.append(galaIMAG_PSFExtra, galaIMAG_PSF[0:galasToAdd])
#             galaZMAG_PSFExtra = np.append(galaZMAG_PSFExtra, galaZMAG_PSF[0:galasToAdd])
#             galaGMAG_CMExtra = np.append(galaGMAG_CMExtra, galaGMAG_CM[0:galasToAdd])
#             galaRMAG_CMExtra = np.append(galaRMAG_CMExtra, galaRMAG_CM[0:galasToAdd])
#             galaIMAG_CMExtra = np.append(galaIMAG_CMExtra, galaIMAG_CM[0:galasToAdd])
#             galaZMAG_CMExtra = np.append(galaZMAG_CMExtra, galaZMAG_CM[0:galasToAdd])
#             galaSIZEExtra = np.append(galaSIZEExtra, galaSIZE[0:galasToAdd])
#             galaSIZE_ERRExtra = np.append(galaSIZE_ERRExtra, galaSIZE_ERR[0:galasToAdd])

#             gala_table =  Table()
#             gala_table['PIX'] = galaPIXExtra
#             gala_table['GMAG_PSF'] = galaGMAG_PSFExtra
#             gala_table['RMAG_PSF'] = galaRMAG_PSFExtra
#             gala_table['IMAG_PSF'] = galaIMAG_PSFExtra
#             gala_table['ZMAG_PSF'] = galaZMAG_PSFExtra
#             gala_table['GMAG_CM'] = galaGMAG_CMExtra
#             gala_table['RMAG_CM'] = galaRMAG_CMExtra
#             gala_table['IMAG_CM'] = galaIMAG_CMExtra
#             gala_table['ZMAG_CM'] = galaZMAG_CMExtra
#             gala_table['SIZE'] = galaSIZEExtra
#             gala_table['SIZE_ERR'] = galaSIZE_ERRExtra
#             writeToGalaFile = equalSizeFileDir + "Galaxies" + ("0" * (4 - len(str(galaIndex)))) + str(galaIndex) + '.fits'
#             gala_table.write(writeToGalaFile, overwrite = True)
#             galaIndex += 1

#             galaPIX = np.delete(galaPIX, np.arange(galasToAdd))
#             galaGMAG_PSF = np.delete(galaGMAG_PSF, np.arange(galasToAdd))
#             galaRMAG_PSF = np.delete(galaRMAG_PSF, np.arange(galasToAdd))
#             galaIMAG_PSF = np.delete(galaIMAG_PSF, np.arange(galasToAdd))
#             galaZMAG_PSF = np.delete(galaZMAG_PSF, np.arange(galasToAdd))
#             galaGMAG_CM = np.delete(galaGMAG_CM, np.arange(galasToAdd))
#             galaRMAG_CM = np.delete(galaRMAG_CM, np.arange(galasToAdd))
#             galaIMAG_CM = np.delete(galaIMAG_CM, np.arange(galasToAdd))
#             galaZMAG_CM = np.delete(galaZMAG_CM, np.arange(galasToAdd))
#             galaSIZE = np.delete(galaSIZE, np.arange(galasToAdd))
#             galaSIZE_ERR = np.delete(galaSIZE_ERR, np.arange(galasToAdd))
#         else:
#             galasToAdd = len(galaPIX)
#             galaPIXExtra = np.append(galaPIXExtra, galaPIX[0:galasToAdd])
#             galaGMAG_PSFExtra = np.append(galaGMAG_PSFExtra, galaGMAG_PSF[0:galasToAdd])
#             galaRMAG_PSFExtra = np.append(galaRMAG_PSFExtra, galaRMAG_PSF[0:galasToAdd])
#             galaIMAG_PSFExtra = np.append(galaIMAG_PSFExtra, galaIMAG_PSF[0:galasToAdd])
#             galaZMAG_PSFExtra = np.append(galaZMAG_PSFExtra, galaZMAG_PSF[0:galasToAdd])
#             galaGMAG_CMExtra = np.append(galaGMAG_CMExtra, galaGMAG_CM[0:galasToAdd])
#             galaRMAG_CMExtra = np.append(galaRMAG_CMExtra, galaRMAG_CM[0:galasToAdd])
#             galaIMAG_CMExtra = np.append(galaIMAG_CMExtra, galaIMAG_CM[0:galasToAdd])
#             galaZMAG_CMExtra = np.append(galaZMAG_CMExtra, galaZMAG_CM[0:galasToAdd])
#             galaSIZEExtra = np.append(galaSIZEExtra, galaSIZE[0:galasToAdd])
#             galaSIZE_ERRExtra = np.append(galaSIZE_ERRExtra, galaSIZE_ERR[0:galasToAdd])

#             galaPIX = np.delete(galaPIX, np.arange(galasToAdd))
#             galaGMAG_PSF = np.delete(galaGMAG_PSF, np.arange(galasToAdd))
#             galaRMAG_PSF = np.delete(galaRMAG_PSF, np.arange(galasToAdd))
#             galaIMAG_PSF = np.delete(galaIMAG_PSF, np.arange(galasToAdd))
#             galaZMAG_PSF = np.delete(galaZMAG_PSF, np.arange(galasToAdd))
#             galaGMAG_CM = np.delete(galaGMAG_CM, np.arange(galasToAdd))
#             galaRMAG_CM = np.delete(galaRMAG_CM, np.arange(galasToAdd))
#             galaIMAG_CM = np.delete(galaIMAG_CM, np.arange(galasToAdd))
#             galaZMAG_CM = np.delete(galaZMAG_CM, np.arange(galasToAdd))
#             galaSIZE = np.delete(galaSIZE, np.arange(galasToAdd))
#             galaSIZE_ERR = np.delete(galaSIZE_ERR, np.arange(galasToAdd))
            
        
#     while len(starPIX) >= starPerFile:
#         star_table =  Table()
#         star_table['PIX'] = starPIX[0:starPerFile]
#         star_table['GMAG_PSF'] = starGMAG_PSF[0:starPerFile]
#         star_table['RMAG_PSF'] = starRMAG_PSF[0:starPerFile]
#         star_table['IMAG_PSF'] = starIMAG_PSF[0:starPerFile]
#         star_table['ZMAG_PSF'] = starZMAG_PSF[0:starPerFile]
#         star_table['GMAG_CM'] = starGMAG_CM[0:starPerFile]
#         star_table['RMAG_CM'] = starRMAG_CM[0:starPerFile]
#         star_table['IMAG_CM'] = starIMAG_CM[0:starPerFile]
#         star_table['ZMAG_CM'] = starZMAG_CM[0:starPerFile]
#         star_table['SIZE'] = starSIZE[0:starPerFile]
#         star_table['SIZE_ERR'] = starSIZE_ERR[0:starPerFile]
#         writeToStarFile = equalSizeFileDir + "Stars" + ("0" * (4 - len(str(starIndex)))) + str(starIndex) + '.fits'
#         star_table.write(writeToStarFile, overwrite = True)
#         starIndex += 1
        
#         starPIX = np.delete(starPIX, np.arange(starPerFile))
#         starGMAG_PSF = np.delete(starGMAG_PSF, np.arange(starPerFile))
#         starRMAG_PSF = np.delete(starRMAG_PSF, np.arange(starPerFile))
#         starIMAG_PSF = np.delete(starIMAG_PSF, np.arange(starPerFile))
#         starZMAG_PSF = np.delete(starZMAG_PSF, np.arange(starPerFile))
#         starGMAG_CM = np.delete(starGMAG_CM, np.arange(starPerFile))
#         starRMAG_CM = np.delete(starRMAG_CM, np.arange(starPerFile))
#         starIMAG_CM = np.delete(starIMAG_CM, np.arange(starPerFile))
#         starZMAG_CM = np.delete(starZMAG_CM, np.arange(starPerFile))
#         starSIZE = np.delete(starSIZE, np.arange(starPerFile))
#         starSIZE_ERR = np.delete(starSIZE_ERR, np.arange(starPerFile))
        
#     while len(galaPIX) >= galaPerFile:
#         gala_table =  Table()
#         gala_table['PIX'] = galaPIX[0:galaPerFile]
#         gala_table['GMAG_PSF'] = galaGMAG_PSF[0:galaPerFile]
#         gala_table['RMAG_PSF'] = galaRMAG_PSF[0:galaPerFile]
#         gala_table['IMAG_PSF'] = galaIMAG_PSF[0:galaPerFile]
#         gala_table['ZMAG_PSF'] = galaZMAG_PSF[0:galaPerFile]
#         gala_table['GMAG_CM'] = galaGMAG_CM[0:galaPerFile]
#         gala_table['RMAG_CM'] = galaRMAG_CM[0:galaPerFile]
#         gala_table['IMAG_CM'] = galaIMAG_CM[0:galaPerFile]
#         gala_table['ZMAG_CM'] = galaZMAG_CM[0:galaPerFile]
#         gala_table['SIZE'] = galaSIZE[0:galaPerFile]
#         gala_table['SIZE_ERR'] = galaSIZE_ERR[0:galaPerFile]
#         writeToGalaFile = equalSizeFileDir + "Galaxies" + ("0" * (4 - len(str(galaIndex)))) + str(galaIndex) + '.fits'
#         gala_table.write(writeToGalaFile, overwrite = True)
#         galaIndex += 1
        
#         galaPIX = np.delete(galaPIX, np.arange(galaPerFile))
#         galaGMAG_PSF = np.delete(galaGMAG_PSF, np.arange(galaPerFile))
#         galaRMAG_PSF = np.delete(galaRMAG_PSF, np.arange(galaPerFile))
#         galaIMAG_PSF = np.delete(galaIMAG_PSF, np.arange(galaPerFile))
#         galaZMAG_PSF = np.delete(galaZMAG_PSF, np.arange(galaPerFile))
#         galaGMAG_CM = np.delete(galaGMAG_CM, np.arange(galaPerFile))
#         galaRMAG_CM = np.delete(galaRMAG_CM, np.arange(galaPerFile))
#         galaIMAG_CM = np.delete(galaIMAG_CM, np.arange(galaPerFile))
#         galaZMAG_CM = np.delete(galaZMAG_CM, np.arange(galaPerFile))
#         galaSIZE = np.delete(galaSIZE, np.arange(galaPerFile))
#         galaSIZE_ERR = np.delete(galaSIZE_ERR, np.arange(galaPerFile))
        
#     if len(starPIX) > 0:
#         starPIXExtra = starPIX
#         starGMAG_PSFExtra = starGMAG_PSF
#         starRMAG_PSFExtra = starRMAG_PSF
#         starIMAG_PSFExtra = starIMAG_PSF
#         starZMAG_PSFExtra = starZMAG_PSF
#         starGMAG_CMExtra = starGMAG_CM
#         starRMAG_CMExtra = starRMAG_CM
#         starIMAG_CMExtra = starIMAG_CM
#         starZMAG_CMExtra = starZMAG_CM
#         starSIZEExtra = starSIZE
#         starSIZE_ERRExtra = starSIZE_ERR
        
#     if len(galaPIX) > 0:
#         galaPIXExtra = galaPIX
#         galaGMAG_PSFExtra = galaGMAG_PSF
#         galaRMAG_PSFExtra = galaRMAG_PSF
#         galaIMAG_PSFExtra = galaIMAG_PSF
#         galaZMAG_PSFExtra = galaZMAG_PSF
#         galaGMAG_CMExtra = galaGMAG_CM
#         galaRMAG_CMExtra = galaRMAG_CM
#         galaIMAG_CMExtra = galaIMAG_CM
#         galaZMAG_CMExtra = galaZMAG_CM
#         galaSIZEExtra = galaSIZE
#         galaSIZE_ERRExtra = galaSIZE_ERR
        
# star_table =  Table()
# star_table['PIX'] = starPIXExtra
# star_table['GMAG_PSF'] = starGMAG_PSFExtra
# star_table['RMAG_PSF'] = starRMAG_PSFExtra
# star_table['IMAG_PSF'] = starIMAG_PSFExtra
# star_table['ZMAG_PSF'] = starZMAG_PSFExtra
# star_table['GMAG_CM'] = starGMAG_CMExtra
# star_table['RMAG_CM'] = starRMAG_CMExtra
# star_table['IMAG_CM'] = starIMAG_CMExtra
# star_table['ZMAG_CM'] = starZMAG_CMExtra
# star_table['SIZE'] = starSIZEExtra
# star_table['SIZE_ERR'] = starSIZE_ERRExtra
# writeToStarFile = equalSizeFileDir + "Stars" + ("0" * (4 - len(str(starIndex)))) + str(starIndex) + '.fits'
# star_table.write(writeToStarFile, overwrite = True)

# gala_table =  Table()
# gala_table['PIX'] = galaPIXExtra
# gala_table['GMAG_PSF'] = galaGMAG_PSFExtra
# gala_table['RMAG_PSF'] = galaRMAG_PSFExtra
# gala_table['IMAG_PSF'] = galaIMAG_PSFExtra
# gala_table['ZMAG_PSF'] = galaZMAG_PSFExtra
# gala_table['GMAG_CM'] = galaGMAG_CMExtra
# gala_table['RMAG_CM'] = galaRMAG_CMExtra
# gala_table['IMAG_CM'] = galaIMAG_CMExtra
# gala_table['ZMAG_CM'] = galaZMAG_CMExtra
# gala_table['SIZE'] = galaSIZEExtra
# gala_table['SIZE_ERR'] = galaSIZE_ERRExtra
# writeToGalaFile = equalSizeFileDir + "Galaxies" + ("0" * (4 - len(str(galaIndex)))) + str(galaIndex) + '.fits'
# star_table.write(writeToGalaFile, overwrite = True)

# Testing

Everything from here on out is just testing for now.

In [24]:
galaTest = fitsio.read(croppedGalasDirectory + '1482.fits')

This means each object takes up a space of about 0.087 kB in memory. Use this to get an 
estimate of the time necessary to run just on this computer by checking how long this takes to run. It takes about 0.23 seconds for each object to run.

In [25]:
galaTest['PIX']

array([2155682, 2155682, 2155682, ..., 2155602, 2155602, 2155602])

In [26]:
quickIndex = np.full(validPix[-1] + 1, validPix[-1] + 1)

In [27]:
quickIndex[validPix] = np.arange(len(validPix))

In [28]:
np.min(validPix[quickIndex[validPix]] == validPix)

True

In [29]:
propMaps = []
for condFile in condFiles:
    condData = fitsio.read(condFile)['SIGNAL'] # This reads in the data
    propMaps.append(condData) # Only stores the values that are in pixels with injections

In [30]:
starY = fitsio.read(starTrainFile)
starAcc = loadtxt(starTrainFile[0:-5] + '_Ave_Acc.csv', delimiter=',')
starInds = loadtxt(starTrainFile[0:-5] + '_Indices.csv', delimiter=',').astype(int)
starMeans = loadtxt(starTrainFile[0:-5] + '_Means.csv', delimiter=',')
starSTDs = loadtxt(starTrainFile[0:-5] + '_Stds.csv', delimiter=',')
starRedSTDs = loadtxt(starTrainFile[0:-5] + '_Red_Stds.csv', delimiter=',')

# This reads in the feature vector and converts it to floating points.
strFeatVec = loadtxt(starTrainFile[0:-5] + '_Feat_Vec.csv', delimiter = '\t', dtype = object)
starFeatVec = []
for j in range(len(strFeatVec)):

    indVec = []

    for k in range(len(strFeatVec[j])):

        indVec.append(float(strFeatVec[j][k]))

    indVec = np.array(indVec)
    starFeatVec.append(indVec)

starFeatVec = np.array(starFeatVec, dtype = object)

In [31]:
galaY = fitsio.read(galaTrainFile)
galaAcc = loadtxt(galaTrainFile[0:-5] + '_Ave_Acc.csv', delimiter=',')
galaInds = loadtxt(galaTrainFile[0:-5] + '_Indices.csv', delimiter=',').astype(int)
galaMeans = loadtxt(galaTrainFile[0:-5] + '_Means.csv', delimiter=',')
galaSTDs = loadtxt(galaTrainFile[0:-5] + '_Stds.csv', delimiter=',')
galaRedSTDs = loadtxt(galaTrainFile[0:-5] + '_Red_Stds.csv', delimiter=',')

# This reads in the feature vector and converts it to floating points.
strFeatVec = loadtxt(galaTrainFile[0:-5] + '_Feat_Vec.csv', delimiter = '\t', dtype = object)
galaFeatVec = []
for j in range(len(strFeatVec)):

    indVec = []

    for k in range(len(strFeatVec[j])):

        indVec.append(float(strFeatVec[j][k]))

    indVec = np.array(indVec)
    galaFeatVec.append(indVec)

galaFeatVec = np.array(galaFeatVec, dtype = object)

In [32]:
testGalaTrainFile = newGalaDir + 'Gala_Test_1-4.fits'

In [33]:
# timeThrough = 0
# for galaxy in galaTest:
#     timeThrough += 1
#     if timeThrough % 100 == 0:
#         print(timeThrough)
#     allProps = []
#     allProps.append(galaxy['GMAG_PSF'])
#     allProps.append(galaxy['RMAG_PSF'])
#     allProps.append(galaxy['IMAG_PSF'])
#     allProps.append(galaxy['ZMAG_PSF'])
#     allProps.append(galaxy['GMAG_CM'])
#     allProps.append(galaxy['RMAG_CM'])
#     allProps.append(galaxy['IMAG_CM'])
#     allProps.append(galaxy['ZMAG_CM'])
#     allProps.append(galaxy['SIZE'])
#     allProps.append(galaxy['SIZE_ERR'])
#     for prop in propMaps:
#         allProps.append(prop[quickIndex[galaxy['PIX']]])
#     check = singleObjectCorrection(allProps, starTrainFile, newGalaDir + 'Gala_Train_1-22.fits', False, starY, starAcc, starInds, starMeans, starSTDs, starRedSTDs, starFeatVec,  galaY, galaAcc, galaInds, galaMeans, galaSTDs, galaRedSTDs, galaFeatVec)